In [1]:
import pandas as pd
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_absolute_error, r2_score
from pathlib import Path 


In [2]:
class ScenicDataset(Dataset):
    def __init__(self, data_frame, root_dir, transform=None):
        self.data_frame = data_frame
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx]['image_path'])
        try:
            image = Image.open(img_name).convert('RGB')
        except (IOError, SyntaxError) as e:
            return None  

        if self.transform:
            image = self.transform(image)

        return image  


In [3]:
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:
from torchvision.models import ResNet50_Weights

class ResNet50FeatureExtractor(nn.Module):
    def __init__(self):
        super(ResNet50FeatureExtractor, self).__init__()
        resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)  
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-1])  

    def forward(self, x):
        with torch.no_grad():
            features = self.feature_extractor(x)
        return features.squeeze()

In [5]:
class RegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x


In [6]:
input_dim = 2048  


feature_extractor = ResNet50FeatureExtractor()
feature_extractor.eval()  

model = RegressionModel(input_dim)
model.load_state_dict(torch.load('/home/ubuntu/landscape-aesthetics/models/resnet50_weights.pth', weights_only=True))
model.eval() 


RegressionModel(
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [7]:
file_location_path = Path.cwd()
project_base_path = file_location_path.parent.parent
data_path = project_base_path / 'data' / 'processed' / 'landscape_score' / 'precessed_data' / 'processed' / 'batch2'
image_folder = Path('/home/ubuntu/landscape-aesthetics')

results_folder = project_base_path / 'data' / 'processed' / 'landscape_score'
results_folder.mkdir(parents=True, exist_ok=True)


In [8]:
def process_all_csv_files(data_path, image_folder, results_folder, feature_extractor, model, input_dim):
    csv_files = Path(data_path).glob('*.csv')
    failed_files = []

    for csv_file in csv_files:
        try:
            
            print(f"\nProcessing {csv_file.name}...", flush=True)
            
            data_frame = pd.read_csv(csv_file)
            print(f"Loaded {len(data_frame)} rows from {csv_file.name}")
            
            dataset = ScenicDataset(data_frame=data_frame, root_dir=image_folder, transform=data_transforms)
            data_loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)
            
            
            predicted_scores = predict(data_loader, feature_extractor, model)
            
            
            results = pd.DataFrame({
                'image_path': data_frame['image_path'],
                'predicted_score': predicted_scores
            })
            
            result_file = results_folder / f'processed_{csv_file.stem}.csv'
            
            results.to_csv(result_file, index=False)
            print(f"Predictions saved to {result_file}")
        
        except Exception as e:
            print(f"Error processing {csv_file.name}: {e}")
            failed_files.append(csv_file.name)

    if failed_files:
        print("The following files failed during processing:")
        for failed_file in failed_files:
            print(failed_file)

In [9]:
def predict(loader, feature_extractor, model):
    model.eval()
    feature_extractor.eval()

    predictions = []

    with torch.no_grad():
        for images in loader:
            if images is None:
                continue
            features = feature_extractor(images)
            outputs = model(features)
            #predictions.extend(outputs.squeeze().cpu().numpy())
            
            if outputs.dim() > 1:
                outputs = outputs.squeeze()
            
            outputs = outputs.cpu().numpy()
            predictions.extend(outputs)

    return predictions


In [10]:
process_all_csv_files(data_path, image_folder, results_folder, feature_extractor, model, input_dim)


Processing ns6_clean_30.csv...


Loaded 45373 rows from ns6_clean_30.csv


Predictions saved to /home/ubuntu/landscape-aesthetics/data/processed/landscape_score/processed_ns6_clean_30.csv

Processing ns6_clean_72.csv...


Loaded 45405 rows from ns6_clean_72.csv


Predictions saved to /home/ubuntu/landscape-aesthetics/data/processed/landscape_score/processed_ns6_clean_72.csv

Processing ns6_clean_61.csv...


...

Predictions saved to /home/ubuntu/landscape-aesthetics/data/processed/landscape_score/processed_ns6_clean_50.csv

Processing ns6_clean_08.csv...


Loaded 45418 rows from ns6_clean_08.csv


Predictions saved to /home/ubuntu/landscape-aesthetics/data/processed/landscape_score/processed_ns6_clean_08.csv

Processing ns6_clean_91.csv...


Loaded 45343 rows from ns6_clean_91.csv


Predictions saved to /home/ubuntu/landscape-aesthetics/data/processed/landscape_score/processed_ns6_clean_91.csv
